In [1]:
# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)


In [2]:
# classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
#            "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# cfg = Config.fromfile('./2ed/85_cascade_swins_fpn_3x_iou_change.py')
# cfg.work_dir = './work_dirs/85_cascade_swins_fpn_3x_iou_change'
# root='../../dataset/'
# cfg.log_config.hooks[1].init_kwargs.name = 'MM_2nd_reappearence'

In [15]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기

# cfg = Config.fromfile('./configs/swin/retinanet_swin-t-p4-w7_fpn_1x_coco.py')
cfg = Config.fromfile('./77_cascade_swins_fpn_3x_TTA.py')##########################################################################
# cfg = Config.fromfile('./2ed/85_cascade_swins_fpn_3x_iou_change.py')
# cfg = Config.fromfile('../SoftTeacher/configs/soft_teacher/soft_teacher_faster_rcnn_r50_caffe_fpn_coco_full_720k.py')

root='../../dataset/'
# root='../../dataset_erase/dataset/'
# root='../../dataset_erase2/dataset/'
print(cfg.model.keys())
# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
cfg.data.train.ann_file = root + 'train.json' # train json 정보
# cfg.data.train.pipeline[2]['img_scale'] = (512,512) # Resize

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
# cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.samples_per_gpu = 4

cfg.seed = 2022
cfg.gpu_ids = [0]
# cfg.work_dir = './work_dirs/4_cascade_swins_fpn_adamW_resize_many'
# cfg.work_dir = './work_dirs/3_MM_nas_fcos_swins_adamW'
# cfg.work_dir = './work_dirs/85_cascade_swins_fpn_3x_iou_change_pseudo'
# cfg.work_dir = './work_dirs/3_soft_teacher_base_720'   #######################################################
cfg.work_dir = './work_dirs/77_cascade_swins_fpn_3x_erase_normal'
# cfg.work_dir = './work_dirs/77_cascade_swins_fpn_3x_erase_erase'
# cfg.work_dir = './work_dirs/77_cascade_swins_fpn_3x_erase_erase_relabelling'

# cfg.model.roi_head.bbox_head.num_classes = 10

cfg.model.roi_head.bbox_head[0].num_classes = 10
cfg.model.roi_head.bbox_head[1].num_classes = 10
cfg.model.roi_head.bbox_head[2].num_classes = 10

# cfg.model.bbox_head.num_classes = 10



cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)

# cfg.log_config.hooks[1].init_kwargs.name = 'MM_cas_rcn_swins_adamW_resizeMany'  #############################3###
# cfg.log_config.hooks[1].init_kwargs.name = 'MM_nas_fcos_swins_adamW'
# cfg.log_config.hooks[1].init_kwargs.name = 'MM_2nd_reappearence'
cfg.log_config.hooks[1].init_kwargs.name = 'erase_test'
# cfg.log_config.hooks[1].init_kwargs.name = 'erase_test_erase'
# cfg.log_config.hooks[1].init_kwargs.name = 'erase_test_erase_relabelling'

dict_keys(['type', 'backbone', 'neck', 'rpn_head', 'roi_head', 'train_cfg', 'test_cfg'])


In [16]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.08s)
creating index...
index created!


In [17]:
#KeyError: 'SemiDataset is not in the dataset registry'
#https://github.com/open-mmlab/mmdetection/issues/3751
#datasets/__init__.py 수정

In [18]:
# dataset 확인
datasets[0]


CocoDataset Train dataset with number of images 4883, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 3965  | 1 [Paper]     | 6352  | 2 [Paper pack]  | 897   | 3 [Metal]   | 936   | 4 [Glass]    | 982   |
| 5 [Plastic]       | 2943  | 6 [Styrofoam] | 1263  | 7 [Plastic bag] | 5178  | 8 [Battery] | 159   | 9 [Clothing] | 468   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [19]:
import warnings

# 경고 메시지를 무시하고 숨기거나
# warnings.filterwarnings(action='ignore')

# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()


# warnings.filterwarnings(action='default')
#https://computer-science-student.tistory.com/275

2022-04-03 00:05:34,254 - mmdet - INFO - load checkpoint from http path: https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_small_patch4_window7_224.pth
2022-04-03 00:05:34,494 - mmdet - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2022-04-03 00:05:34,517 - mmdet - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2022-04-03 00:05:34,526 - mmdet - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type': 'Xavier', 'distribution': 'uniform', 'override': [{'name': 'shared_fcs'}, {'name': 'cls_fcs'}, {'name': 'reg_fcs'}]}]
2022-04-03 00:05:34,628 - mmdet - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type

In [20]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=False)

2022-04-03 00:05:35,373 - mmdet - INFO - Start running, host: root@71b0d7392d91, work_dir: /opt/ml/detection/baseline/mmdetection/work_dirs/77_cascade_swins_fpn_3x_erase_normal
2022-04-03 00:05:35,374 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
 -------------------- 
after_train_iter:
(ABOVE_NORMAL) OptimizerHook        

2022-04-03 00:13:17,463 - mmdet - INFO - Epoch [1][400/1221]	lr: 7.982e-05, eta: 3:44:33, time: 1.141, data_time: 0.016, memory: 19397, loss_rpn_cls: 0.1732, loss_rpn_bbox: 0.0381, s0.loss_cls: 0.3966, s0.acc: 90.3344, s0.loss_bbox: 0.1418, s1.loss_cls: 0.1321, s1.acc: 95.4756, s1.loss_bbox: 0.0535, s2.loss_cls: 0.0552, s2.acc: 95.5709, s2.loss_bbox: 0.0100, loss: 1.0004, grad_norm: 5.8139
2022-04-03 00:20:50,377 - mmdet - INFO - Epoch [1][800/1221]	lr: 1.000e-04, eta: 3:36:08, time: 1.132, data_time: 0.010, memory: 19397, loss_rpn_cls: 0.0511, loss_rpn_bbox: 0.0336, s0.loss_cls: 0.2563, s0.acc: 93.5155, s0.loss_bbox: 0.1376, s1.loss_cls: 0.1076, s1.acc: 94.6035, s1.loss_bbox: 0.0881, s2.loss_cls: 0.0412, s2.acc: 96.0460, s2.loss_bbox: 0.0325, loss: 0.7478, grad_norm: 3.4849
2022-04-03 00:28:20,371 - mmdet - INFO - Epoch [1][1200/1221]	lr: 1.000e-04, eta: 3:27:51, time: 1.125, data_time: 0.010, memory: 19397, loss_rpn_cls: 0.0445, loss_rpn_bbox: 0.0311, s0.loss_cls: 0.2509, s0.acc: 93.

learning_rate,▃▆██████████████████████████████▁▁▁▁▁▁▁▁
momentum,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/grad_norm,█▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▃▃▂▂▂▃▃▃▂▃▃▃▂▂▂▂▂▂▂▂▂
train/loss,█▃▃▃▃▃▃▃▃▃▃▂▃▂▃▂▃▂▂▂▂▂▃▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▂
train/loss_rpn_bbox,█▆▆▅▅▅▄▅▄▄▄▃▃▃▄▃▃▃▃▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▁▂▂▁▁▂
train/loss_rpn_cls,█▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/s0.acc,▁▆▆▆▆▆▆▆▆▆▆▇▇▇▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇████▇
train/s0.loss_bbox,▄█▅▅▄▅▄▄▄▄▄▃▃▃▃▂▃▃▃▃▃▃▃▂▃▃▃▂▂▂▃▂▂▂▂▂▁▂▁▂
train/s0.loss_cls,█▄▃▃▃▃▃▃▂▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
train/s1.acc,▄█▃▂▂▁▂▁▂▂▂▃▂▃▂▃▂▄▃▃▂▃▂▃▃▃▃▃▃▄▂▃▅▆▅▅▆▆▆▅
train/s1.loss_bbox,▁▄▇▇██▇█▇▇█▆▆▆▇▆▇▆▇▆▆▆▇▆▇▆▆▆▆▆▇▇▆▅▆▆▅▆▅▆
